# Introduction to py-sdk

The python sdk allows you to create queries and transaction to a Nibiru chain.

It allows to interact with all of the modules and can be leveraged to automate trading strategies or monitor them.

This notebook will guide you on opening and closing positions.

In [ ]:
import json

from nibiru import Network, ChainClient, Msg, Coin

Let's set up the chain to which we will be connecting to. If you have a preferred endpoint you can update these values to connect to it. Make sure that the node have the port opens for 1317, 26657 and 9090.

In [ ]:
pair = "ubtc:unusd"

Next, let's make sure that the transaction we send are synchronous. This way if no error is raised when running the execute_msgs method, we are sure that the transaction was successful.

In [ ]:
mnemonic = "exclude rural knife asset egg seminar carry control thought stock topic lens embark about huge obvious reason language ripple clean cousin library punch renew"
client = ChainClient(network=Network.testnet(2))
client.authenticate(mnemonic=mnemonic)

# Check balance
print(client.query.get_bank_balances(client.address))

Now with those fresh token, we can open a long position

In [ ]:
output = client.tx.execute_msgs(
    Msg.perp.open_position(
        pair=pair,
        is_long=True,
        quote_asset_amount=10,
        leverage=1,
        base_asset_amount_limit=0,
    )
)
print(output)

If your TX has succeeded, you should get output something like:
ExecuteTxResp(code=0, tx_hash='946A4252423A0AC183C1BAA2B76CCB25512DC9A18861D8782EFE7F14512CBDF6', log='[]')
Query your TX hash to get TX details:

In [ ]:
output = client.query.tx_by_hash("946A4252423A0AC183C1BAA2B76CCB25512DC9A18861D8782EFE7F14512CBDF6")
print(output)

Or directly query the state of your position:

In [ ]:
position = client.query.perp.position(pair, client.address)
print(json.dumps(position, indent=4))

We can then add margin to our position:

In [ ]:
output = client.tx.execute_msgs(
    Msg.perp.add_margin(
        pair=pair,
        margin=Coin(1, "unusd"),
    )
)
print(output)

Or remove margin:

In [ ]:
output = client.tx.execute_msgs(
    Msg.perp.remove_margin(
        pair=pair,
        margin=Coin(1, "unusd")
    )
)
print(output)

We can then totally close the position:

In [ ]:
output = client.tx.execute_msgs(
    Msg.perp.close_position(
        pair=pair,
    )
)
print(output)

## Multi-transactions

We can build multiple messages into a single transactions to be sure that they are executed consecutively.

It can be useful for example to send tokens after removing margins from a position

Lets create a transction with:
- Agent open a position
- Agent add margin
- Agent close the position

In [ ]:
output = client.tx.execute_msgs(
    [
        Msg.perp.open_position(
            pair=pair,
            is_long=True,
            quote_asset_amount=10,
            leverage=1,
            base_asset_amount_limit=0,
        ),
        Msg.perp.add_margin(
            pair=pair,
            margin=Coin(1, "unusd"),
        ),
        Msg.perp.close_position(
            pair=pair,
        ),
    ]
)
print(output)